# Mapeamento de patentes

## Bibliotecas

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import requests
import zipfile
import time

## Download das revistas em zip (com tratamento de exceção)

Fonte de dados: http://revistas.inpi.gov.br/rpi/#

In [3]:
#Funcao para verificar se o arquivo zip ja foi baixado anteriormente.
def download_zip(zip_url, output_folder):
    # Nome do arquivo na pasta
    file_name = os.path.basename(zip_url)

    # Caminho completo para o arquivo ZIP
    zip_path = os.path.join(output_folder, file_name)

    # Verifique se o arquivo já existe na pasta
    if os.path.exists(zip_path):
        print(f"Arquivo ZIP '{file_name}' já existe na pasta, pulando o download.")
        return

    # Download do arquivo ZIP diretamente na pasta de saída + TRATAMENTO DE EXCEÇÃO
    max_retries = 3  # Número máximo de tentativas
    for retry in range(max_retries):
        try:
            response = requests.get(zip_url)
            response.raise_for_status()  # Lançar exceção se a resposta não for bem-sucedida

            with open(zip_path, "wb") as zip_file:
                zip_file.write(response.content)

            print(f"Arquivo ZIP salvo em '{zip_path}'")
            break  # Download bem-sucedido, saia do loop de tentativas
        except requests.exceptions.RequestException as e:
            if retry < max_retries - 1:
                if isinstance(e, requests.exceptions.HTTPError) and e.response.status_code == 404:
                    print("Erro: Arquivo não encontrado.")
                    break  # Não é necessário tentar novamente se o arquivo não existe
                else:
                    print(f"Erro ao baixar o arquivo ZIP '{file_name}': {e}")
                    time.sleep(5)  # Aguarde 5 segundos antes de tentar novamente
            else:
                print(f"Erro ao baixar o arquivo ZIP '{file_name}': {e}")
                break

url2 = 2557
while url2 <= 2660:
    zip_url = 'http://revistas.inpi.gov.br/txt/P' + str(url2) + '.zip'
    # Pasta onde você deseja salvar o arquivo ZIP
    output_folder = 'dados_zip'

    # Verifique se a pasta de saída existe e a crie se não existir
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    download_zip(zip_url, output_folder)
    url2 += 1


Arquivo ZIP salvo em 'dados_zip\P2557.zip'
Arquivo ZIP salvo em 'dados_zip\P2558.zip'
Arquivo ZIP salvo em 'dados_zip\P2559.zip'
Arquivo ZIP salvo em 'dados_zip\P2560.zip'
Arquivo ZIP salvo em 'dados_zip\P2561.zip'
Arquivo ZIP salvo em 'dados_zip\P2562.zip'
Arquivo ZIP salvo em 'dados_zip\P2563.zip'
Arquivo ZIP salvo em 'dados_zip\P2564.zip'
Arquivo ZIP salvo em 'dados_zip\P2565.zip'
Arquivo ZIP salvo em 'dados_zip\P2566.zip'
Arquivo ZIP salvo em 'dados_zip\P2567.zip'
Arquivo ZIP salvo em 'dados_zip\P2568.zip'
Arquivo ZIP salvo em 'dados_zip\P2569.zip'
Arquivo ZIP salvo em 'dados_zip\P2570.zip'
Arquivo ZIP salvo em 'dados_zip\P2571.zip'
Arquivo ZIP salvo em 'dados_zip\P2572.zip'
Arquivo ZIP salvo em 'dados_zip\P2573.zip'
Arquivo ZIP salvo em 'dados_zip\P2574.zip'
Arquivo ZIP salvo em 'dados_zip\P2575.zip'
Arquivo ZIP salvo em 'dados_zip\P2576.zip'
Arquivo ZIP salvo em 'dados_zip\P2577.zip'
Arquivo ZIP salvo em 'dados_zip\P2578.zip'
Arquivo ZIP salvo em 'dados_zip\P2579.zip'
Arquivo ZIP

## Descompactar e organizar os arquivos

In [4]:
def extract_zip_files(output_folder):
    # Certifique-se de que a pasta de saída existe
    if not os.path.exists(output_folder):
        print(f"A pasta de saída '{output_folder}' não existe.")
        return

    # Pasta onde você deseja salvar os arquivos XML extraídos
    xml_folder = 'dados_xml'

    # Verifique se a pasta de saída para os XML existe e a crie se não existir
    if not os.path.exists(xml_folder):
        os.makedirs(xml_folder)

    # Percorre todos os arquivos na pasta de saída
    for root, dirs, files in os.walk(output_folder):
        for file in files:
            if file.lower().endswith(".zip"):
                zip_path = os.path.join(root, file)

                try:
                    # Abra o arquivo ZIP
                    with zipfile.ZipFile(zip_path, "r") as zipf:
                        for filename in zipf.namelist():
                            if filename.startswith("Patente_"):
                                # Extraia o arquivo XML correspondente
                                with zipf.open(filename) as xml_file:
                                    xml_content = xml_file.read()
                                    xml_filename = os.path.join(xml_folder, filename)
                                    with open(xml_filename, "wb") as output_file:
                                        output_file.write(xml_content)

                                # Move o arquivo XML extraído com sucesso para a pasta 'dados_xml'
                                os.rename(xml_filename, os.path.join(xml_folder, filename))
                                print(f"Arquivo XML {filename} extraído com sucesso em '{xml_filename}' e movido para '{xml_folder}'")
                except zipfile.BadZipFile:
                    print(f"Erro ao extrair o arquivo ZIP '{file}': Arquivo ZIP inválido.")
                except Exception as e:
                    print(f"Erro ao processar o arquivo ZIP '{file}': {e}")

# Exemplo de uso
extract_zip_files('dados_zip')


Arquivo XML Patente_2557_07012020.xml extraído com sucesso em 'dados_xml\Patente_2557_07012020.xml' e movido para 'dados_xml'
Arquivo XML Patente_2558_14012020.xml extraído com sucesso em 'dados_xml\Patente_2558_14012020.xml' e movido para 'dados_xml'
Arquivo XML Patente_2559_21012020.xml extraído com sucesso em 'dados_xml\Patente_2559_21012020.xml' e movido para 'dados_xml'
Arquivo XML Patente_2560_28012020.xml extraído com sucesso em 'dados_xml\Patente_2560_28012020.xml' e movido para 'dados_xml'
Arquivo XML Patente_2561_04022020.xml extraído com sucesso em 'dados_xml\Patente_2561_04022020.xml' e movido para 'dados_xml'
Arquivo XML Patente_2562_11022020.xml extraído com sucesso em 'dados_xml\Patente_2562_11022020.xml' e movido para 'dados_xml'
Arquivo XML Patente_2563_18022020.xml extraído com sucesso em 'dados_xml\Patente_2563_18022020.xml' e movido para 'dados_xml'
Arquivo XML Patente_2564_27022020.xml extraído com sucesso em 'dados_xml\Patente_2564_27022020.xml' e movido para 'dad

## Raspagem dos dados

In [5]:
# Definir função para raspagem de um arquivo xml
def raspagem(soup):
    # Inicializar listas para armazenar os dados
    data_publicacao = []
    diretoria = []
    numero_revista = []
    codigo = []
    titulo = []
    numero = []
    data_deposito = []
    data_prioridade = []
    cod_prioridade = []
    classificacoes_internacionais = []
    classificacoes_nacionais = []
    titulo_inid = []
    titular_nome = []
    titular_uf = []
    titular_pais = []
    inventor_nome = []

    # Extrair os dados do xml
    for despacho in soup.find_all(lambda tag: tag.name == 'despacho' and '<codigo>3.1</codigo>' in str(tag)):
        data_publ = soup.revista['dataPublicacao']
        diret = soup.revista['diretoria']
        num_rev = soup.revista['numero']
        cod = despacho.find('codigo').text
        tit = despacho.find('titulo').text
        num = despacho.find('numero').text
        data = despacho.find('data-deposito').text
        data_prio = despacho.find('data-prioridade', inid="31").text if despacho.find('data-prioridade', inid="31") else None
        cod_prio1 = despacho.find('sigla-pais', inid="33").text if despacho.find('sigla-pais', inid="33") else None
        cod_prio2 = despacho.find('numero-prioridade', inid="32").text if despacho.find('numero-prioridade', inid="32") else None
        cod_prio = cod_prio1 + cod_prio2 if (cod_prio1 != None and cod_prio2 != None) else None
        internacionais = '; '.join(ci.text for ci in despacho.find_all('classificacao-internacional'))
        nacionais = '; '.join(cn.text for cn in despacho.find_all('classificacao-nacional'))
        tit_inid = despacho.find('titulo', inid="54").text if despacho.find('titulo', inid="54") else None
        titu_nome = despacho.find('nome-completo').text if despacho.find('nome-completo') else None
        titu_uf = despacho.find('uf').text if despacho.find('uf') else None
        titu_pais = despacho.find('sigla').text if despacho.find('sigla') else None
        invent_nome = despacho.find('inventor').text if despacho.find('inventor') else None
        # Incluir os dados na lista
        data_publicacao.append(data_publ)
        diretoria.append(diret)
        numero_revista.append(num_rev)
        codigo.append(cod)
        titulo.append(tit)
        numero.append(num)
        data_deposito.append(data)
        data_prioridade.append(data_prio)
        cod_prioridade.append(cod_prio)
        classificacoes_internacionais.append(internacionais)
        classificacoes_nacionais.append(nacionais)
        titulo_inid.append(tit_inid)
        titular_nome.append(titu_nome)
        titular_uf.append(titu_uf)
        titular_pais.append(titu_pais)
        inventor_nome.append(invent_nome)
        # Transformar listas em dataframe
        df = pd.DataFrame({
            'data_publicacao' : data_publicacao,
            'diretoria' : diretoria,
            'numero_revista' : numero_revista,
            'codigo' : codigo,
            'titulo' : titulo,
            'numero' : numero,
            'data_deposito' : data_deposito,
            'data_prioridade' : data_prioridade,
            'cod_prioridade' : cod_prioridade,
            'classif_internacion' : classificacoes_nacionais,
            'classif_nacion' : classificacoes_nacionais,
            'titulo_inid' : titulo_inid,
            'titular_nome' : titular_nome,
            'titular_uf' : titular_uf,
            'titular_pais' : titular_pais,
            'inventor_nome' : inventor_nome
            })
    return(df)

## Salvar dataframe completo

In [9]:
!pip install tqdm --quiet


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from tqdm import tqdm

In [11]:
# Inicializar o dataframe
data = pd.DataFrame()

# Inicializar o timer
start_time = time.time()

# Lista de arquivos
arquivos = os.listdir('dados_xml')

# Inicializar a barra de progresso
with tqdm(total=len(arquivos), desc="Processando arquivos") as pbar:
    for file in arquivos:
        filepath = os.path.join('dados_xml', file)
        with open(filepath, encoding='utf-8') as arquivo_xml:
            conteudo = arquivo_xml.read()
        soup = BeautifulSoup(conteudo, 'xml')
        df = raspagem(soup)
        data = pd.concat([data, df], axis=0)
        pbar.update(1)

# Finalizar o timer
end_time = time.time()

# Calcular o tempo decorrido
tempo_decorrido = end_time - start_time

data.head(5)  # Verificar o dataframe

print(f"Tempo decorrido: {tempo_decorrido:.2f} segundos")


Processando arquivos: 100%|████████████████| 104/104 [1:00:27<00:00, 34.88s/it]

Tempo decorrido: 3627.97 segundos


### Verificar informações do dataframe completo

In [13]:
data.head(5)

,data_publicacao,diretoria,numero_revista,codigo,titulo,numero,data_deposito,data_prioridade,cod_prioridade,classif_internacion,classif_nacion,titulo_inid,titular_nome,titular_uf,titular_pais,inventor_nome
0,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2013 016644-8,27/06/2013,None,None,B29B 17/00; B29C 43/00,B29B 17/00; B29C 43/00,APERFEIÇOAMENTO INTRODUZIDO EM PROCESSO DE REC...,ACM EL ORRA COLCHÕES ME,SP,BR,\nABDO CARIM MOUMTEZ EL ORRA\n
1,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2014 001105-6,16/01/2014,18/01/2013,TW102201214,F16C 11/04; E05F 1/1215,F16C 11/04; E05F 1/1215,DOBRADIÇA PARA CONJUNTO DE CAPAS PROTETORAS,"KEYARROW (TAIWAN) CO., LTD.",None,TW,\nFU-YI CHEN\n
2,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 004929-9,13/03/2017,None,None,H01Q 13/106; H01Q 1/241,H01Q 13/106; H01Q 1/241,"UMA NOVA ANTENA DE MICROFITA TRI-BANDA, DE TAM...",UNIVERSIDADE FEDERAL DO CEARÁ,CE,BR,\nANTONIO SERGIO BEZERRA SOMBRA\n
3,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 024659-0,17/11/2017,None,None,G01F 1/58; H04W 4/12,G01F 1/58; H04W 4/12,DISPOSITIVO DE MÚLTIPLOS MÓDULOS COM OTIMIZAÇÃ...,CID GRANJA DOS SANTOS,None,BR,\nCID GRANJA DOS SANTOS\n
4,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 026178-6,05/12/2017,None,None,H04L 67/42; H04L 61/1558; H04L 29/12141,H04L 67/42; H04L 61/1558; H04L 29/12141,BEHAVIOR DEFINITION MODEL FOR INTERNET OF THIN...,FUNDAÇÃO UNIVERSIDADE DO AMAZONAS,AM,BR,\nEDUARDO JAMES PEREIRA SOUTO\n


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14098 entries, 0 to 77
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   data_publicacao      14098 non-null  object
 1   diretoria            14098 non-null  object
 2   numero_revista       14098 non-null  object
 3   codigo               14098 non-null  object
 4   titulo               14098 non-null  object
 5   numero               14098 non-null  object
 6   data_deposito        14098 non-null  object
 7   data_prioridade      3031 non-null   object
 8   cod_prioridade       3031 non-null   object
 9   classif_internacion  14098 non-null  object
 10  classif_nacion       14098 non-null  object
 11  titulo_inid          14098 non-null  object
 12  titular_nome         14098 non-null  object
 13  titular_uf           10921 non-null  object
 14  titular_pais         14098 non-null  object
 15  inventor_nome        14098 non-null  object
dtypes: object(16

## Tratamento dos dados

In [15]:
#Tratamento dos dados da coluna inventor_nome
data['inventor_nome'] = data['inventor_nome'].str.replace('\n', '') #Para tratar os dados da coluna inventor_nome

In [36]:
# Substituir todos os valores nulos por "NaN"
data_substituido = data.fillna("NaN")
print(data_substituido)

   data_publicacao diretoria numero_revista codigo  \
0       07/01/2020   Patente           2557    3.1   
1       07/01/2020   Patente           2557    3.1   
2       07/01/2020   Patente           2557    3.1   
3       07/01/2020   Patente           2557    3.1   
4       07/01/2020   Patente           2557    3.1   
..             ...       ...            ...    ...   
73      28/12/2021   Patente           2660    3.1   
74      28/12/2021   Patente           2660    3.1   
75      28/12/2021   Patente           2660    3.1   
76      28/12/2021   Patente           2660    3.1   
77      28/12/2021   Patente           2660    3.1   

                                               titulo               numero  \
0   Publicação do Pedido de Patente ou de Certific...  BR 10 2013 016644-8   
1   Publicação do Pedido de Patente ou de Certific...  BR 10 2014 001105-6   
2   Publicação do Pedido de Patente ou de Certific...  BR 10 2017 004929-9   
3   Publicação do Pedido de Patente ou 

In [37]:
data_substituido.head(10)

,data_publicacao,diretoria,numero_revista,codigo,titulo,numero,data_deposito,data_prioridade,cod_prioridade,classif_internacion,classif_nacion,titulo_inid,titular_nome,titular_uf,titular_pais,inventor_nome
0,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2013 016644-8,27/06/2013,NaN,NaN,B29B 17/00; B29C 43/00,B29B 17/00; B29C 43/00,APERFEIÇOAMENTO INTRODUZIDO EM PROCESSO DE REC...,ACM EL ORRA COLCHÕES ME,SP,BR,ABDO CARIM MOUMTEZ EL ORRA
1,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2014 001105-6,16/01/2014,18/01/2013,TW102201214,F16C 11/04; E05F 1/1215,F16C 11/04; E05F 1/1215,DOBRADIÇA PARA CONJUNTO DE CAPAS PROTETORAS,"KEYARROW (TAIWAN) CO., LTD.",NaN,TW,FU-YI CHEN
2,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 004929-9,13/03/2017,NaN,NaN,H01Q 13/106; H01Q 1/241,H01Q 13/106; H01Q 1/241,"UMA NOVA ANTENA DE MICROFITA TRI-BANDA, DE TAM...",UNIVERSIDADE FEDERAL DO CEARÁ,CE,BR,ANTONIO SERGIO BEZERRA SOMBRA
3,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 024659-0,17/11/2017,NaN,NaN,G01F 1/58; H04W 4/12,G01F 1/58; H04W 4/12,DISPOSITIVO DE MÚLTIPLOS MÓDULOS COM OTIMIZAÇÃ...,CID GRANJA DOS SANTOS,NaN,BR,CID GRANJA DOS SANTOS
4,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 026178-6,05/12/2017,NaN,NaN,H04L 67/42; H04L 61/1558; H04L 29/12141,H04L 67/42; H04L 61/1558; H04L 29/12141,BEHAVIOR DEFINITION MODEL FOR INTERNET OF THIN...,FUNDAÇÃO UNIVERSIDADE DO AMAZONAS,AM,BR,EDUARDO JAMES PEREIRA SOUTO
5,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 027316-4,06/12/2017,NaN,NaN,A23L 29/231; A23L 2/04; A23L 19/00; C08B 37/00...,A23L 29/231; A23L 2/04; A23L 19/00; C08B 37/00...,MÉTODO DE EXTRAÇÃO DA PECTINA ADAPTADA PARA SE...,MISSÃO SALESIANA DE MATO GROSSO UNIVERSIDADE C...,MS,BR,MARNEY PASCOLI CEREDA
6,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 001913-9,29/01/2018,NaN,NaN,C08G 18/4277; C08K 2003/0856; C08K 3/10; A43C ...,C08G 18/4277; C08K 2003/0856; C08K 3/10; A43C ...,COMPÓSITO INDUTÍVEL ELETROMAGNETICAMENTE CONTE...,GRENDENE S.A.,CE,BR,EDUARDO RADAELLI PASQUAL
7,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 009789-0,15/05/2018,NaN,NaN,G06Q 30/00; G06F 3/04842; G06Q 10/08; G07C 9/0...,G06Q 30/00; G06F 3/04842; G06Q 10/08; G07C 9/0...,MÁQUINA DE COMPARTILHAMENTO E ALUGUEL DE OBJETOS,RENTBRELLA S.A.,SP,BR,"Sigilo de Autor, conforme solicitado e com bas..."
8,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 009796-2,15/05/2018,NaN,NaN,,,DESENVOLVIMENTO DE PRODUTOS FITOTERÁPICOS AD...,FUNDAÇÃO UNIVERSIDADE ESTADUAL DO CEARÁ - FUNECE,CE,BR,JOSÉ CLAUDIO CARNEIRO DE FREITAS
9,07/01/2020,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2018 011938-9,13/06/2018,NaN,NaN,H04M 1/20; H04B 1/3888; H04M 1/035,H04M 1/20; H04B 1/3888; H04M 1/035,ESTÚDIO PORTÁTIL COM ISOLAMENTO ACÚSTICO PARA ...,MARCUS CEZAR KLAUTAU BONNA,SP,BR,MARCUS CEZAR KLAUTAU BONNA


In [33]:
data_substituido.head(5)

,data_publicacao,diretoria,numero_revista,codigo,titulo,numero,data_deposito,data_prioridade,cod_prioridade,classif_internacion,classif_nacion,titulo_inid,titular_nome,titular_uf,titular_pais,inventor_nome
0,2020-01-07,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2013 016644-8,2013-06-27,NaT,NaN,B29B 17/00; B29C 43/00,B29B 17/00; B29C 43/00,APERFEIÇOAMENTO INTRODUZIDO EM PROCESSO DE REC...,ACM EL ORRA COLCHÕES ME,SP,BR,ABDO CARIM MOUMTEZ EL ORRA
1,2020-01-07,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2014 001105-6,2014-01-16,2013-01-18,TW102201214,F16C 11/04; E05F 1/1215,F16C 11/04; E05F 1/1215,DOBRADIÇA PARA CONJUNTO DE CAPAS PROTETORAS,"KEYARROW (TAIWAN) CO., LTD.",NaN,TW,FU-YI CHEN
2,2020-01-07,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 004929-9,2017-03-13,NaT,NaN,H01Q 13/106; H01Q 1/241,H01Q 13/106; H01Q 1/241,"UMA NOVA ANTENA DE MICROFITA TRI-BANDA, DE TAM...",UNIVERSIDADE FEDERAL DO CEARÁ,CE,BR,ANTONIO SERGIO BEZERRA SOMBRA
3,2020-01-07,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 024659-0,2017-11-17,NaT,NaN,G01F 1/58; H04W 4/12,G01F 1/58; H04W 4/12,DISPOSITIVO DE MÚLTIPLOS MÓDULOS COM OTIMIZAÇÃ...,CID GRANJA DOS SANTOS,NaN,BR,CID GRANJA DOS SANTOS
4,2020-01-07,Patente,2557,3.1,Publicação do Pedido de Patente ou de Certific...,BR 10 2017 026178-6,2017-12-05,NaT,NaN,H04L 67/42; H04L 61/1558; H04L 29/12141,H04L 67/42; H04L 61/1558; H04L 29/12141,BEHAVIOR DEFINITION MODEL FOR INTERNET OF THIN...,FUNDAÇÃO UNIVERSIDADE DO AMAZONAS,AM,BR,EDUARDO JAMES PEREIRA SOUTO


In [38]:
# Salvar o DataFrame em um arquivo CSV
data_substituido.to_csv('data_patentes.csv', index=False)

In [39]:
# Exporte o DataFrame para o arquivo Excel
data.to_excel('data_patentes_excel.xlsx', sheet_name='Planilha Patentes', index=False)